In [24]:
import pandas as pd
import numpy as np
import requests
import time
import re

## Getting our API

In [2]:
with open('movie-api.txt', 'r') as file:
    apikey = file.read()

In [25]:
base_url = "https://api.themoviedb.org/3/movie/"
pop_reviews = "movie/reviews"
url = f"{base_url}?api_key={apikey}"

movie_data = []

for movie_id in range(1, 50):  # Adjust this range as needed
    # Get movie details
    movie_url = f"{base_url}{movie_id}?api_key={apikey}"
    r = requests.get(movie_url)
    
    if r.ok:
        movie = r.json()
        
        # Prepare movie details
        movie_details = {
            'id': movie.get('id'),
            'title': movie.get('title'),
            'release_date': movie.get('release_date'),
            'revenue': movie.get('revenue'),
            'budget': movie.get('budget'),
            'production_companies': ', '.join([company['name'] for company in movie.get('production_companies', [])]),
            'genres': ', '.join([genre['name'] for genre in movie.get('genres', [])]),
            'popularity': movie.get('popularity'),
            'vote_average': movie.get('vote_average'),
            'vote_count': movie.get('vote_count'),
            'overview': movie.get('overview')
        }

        # Get movie reviews
        reviews_url = f"{base_url}{movie_id}/reviews?api_key={apikey}"
        reviews_response = requests.get(reviews_url)
        
        if reviews_response.ok:
            reviews_data = reviews_response.json()
            reviews = reviews_data.get('results', [])
            
            # Get all movie reviews and combine them into one long review
            review_texts = [review['content'] for review in reviews]
            movie_details['reviews'] = ' '.join(review_texts) if review_texts else 'No reviews available'
        else:
            movie_details['reviews'] = 'No reviews available'
        
        movie_data.append(movie_details)
    
    # Add a delay to prevent overwhelming the API
    time.sleep(1)

# Convert to data frame
movies_df = pd.DataFrame(movie_data)
movies_df.head(20)


    id                                              title release_date  \
0    2                                              Ariel   1988-10-21   
1    3                                Shadows in Paradise   1986-10-17   
2    5                                         Four Rooms   1995-12-09   
3    6                                     Judgment Night   1993-10-15   
4    8                   Life in Loops (A Megacities RMX)   2006-01-01   
5    9                                   Sunday in August   2004-09-02   
6   11                                          Star Wars   1977-05-25   
7   12                                       Finding Nemo   2003-05-30   
8   13                                       Forrest Gump   1994-06-23   
9   14                                    American Beauty   1999-09-15   
10  15                                       Citizen Kane   1941-04-17   
11  16                                 Dancer in the Dark   2000-09-01   
12  17                                

In [26]:
movies_df.head(20)

,id,title,release_date,revenue,budget,production_companies,genres,popularity,vote_average,vote_count,overview,reviews
0,2,Ariel,1988-10-21,0,0,Villealfa Filmproductions,"Comedy, Drama, Romance, Crime",9.571,7.118,331,After the coal mine he works at closes and his...,No reviews available
1,3,Shadows in Paradise,1986-10-17,0,0,Villealfa Filmproductions,"Comedy, Drama, Romance",6.986,7.301,392,"Nikander, a rubbish collector and would-be ent...","Released in 1986, Aki Kaurismaki's <i>Varjoja ..."
2,5,Four Rooms,1995-12-09,4257354,4000000,"Miramax, A Band Apart",Comedy,14.295,5.846,2618,It's Ted the Bellhop's first night on the job....,No reviews available
3,6,Judgment Night,1993-10-15,12136938,21000000,"Largo Entertainment, JVC, Universal Pictures","Action, Crime, Thriller",12.757,6.500,331,"Four young friends, while taking a shortcut en...",No reviews available
4,8,Life in Loops (A Megacities RMX),2006-01-01,0,42000,inLoops,Documentary,3.206,7.300,28,Timo Novotny labels his new project an experim...,No reviews available
5,9,Sunday in August,2004-09-02,0,0,,Drama,2.797,7.100,27,,No reviews available
6,11,Star Wars,1977-05-25,775398007,11000000,"Lucasfilm Ltd., 20th Century Fox","Adventure, Action, Science Fiction",86.625,8.204,20445,Princess Leia is captured and held hostage by ...,"(As I'm writing this review, Darth Vader's the..."
7,12,Finding Nemo,2003-05-30,940335536,94000000,Pixar,"Animation, Family",72.214,7.819,19081,"Nemo, an adventurous young clownfish, is unexp...",One of the best animated films I have ever see...
8,13,Forrest Gump,1994-06-23,677387716,55000000,"Paramount Pictures, The Steve Tisch Company, W...","Comedy, Drama, Romance",82.116,8.500,27220,A man with a low IQ has accomplished great thi...,Best movie ever. This is the best movie ever! ...
9,14,American Beauty,1999-09-15,356296601,15000000,"DreamWorks Pictures, Jinks/Cohen Company",Drama,30.794,8.019,11913,"Lester Burnham, a depressed suburban father in...",The film American Beauty to me is a film about...
